## WebScrapping to Alkosto (Online Store)

### 1. Library Import


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import logging
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import time

### 2. Automatización de Extracción de Datos de Televisores Usando Selenium

In [ ]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='scraping.log', filemode='w')

def configure_driver():
    """Configura el WebDriver para usar Chrome."""
    options = Options()
    options.add_argument("--headless")  # Ejecutar en modo sin cabeza (sin interfaz gráfica)
    options.add_argument("--no-sandbox")  # Opcional: para evitar problemas en entornos limitados
    options.add_argument("--disable-dev-shm-usage")  # Opcional: para evitar problemas de memoria
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def load_all_tv_items(driver):
    """Cargar hasta 2 veces los elementos de TV haciendo clic en 'mostrar más'."""
    max_clicks = 7
    clicks = 0

    while clicks < max_clicks:
        try:
            # Espera a que el botón de 'mostrar más' esté presente y visible
            show_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".js-load-more"))
            )
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)  # Esperar un poco para que se carguen más elementos
            clicks += 1  # Incrementar el contador de clics
        except TimeoutException:
            logging.info("No hay más productos para cargar.")
            break  # Sale del bucle si el botón no está disponible

    logging.info(f"Se hicieron {clicks} clics en 'mostrar más'.")

def scrape_tv_data(driver, url):
    """Extrae datos de TVs desde el sitio web de Alkosto."""
    driver.get(url)
    data = []

    try:
        # Cargar todos los elementos disponibles
        load_all_tv_items(driver)

        # Esperar hasta que todos los elementos de TV estén presentes
        li_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#js-hits li"))
        )

        for element in li_elements:
            try:
                # Intentar extraer los elementos necesarios
                title_element = element.find_element(By.TAG_NAME, "h3")
                price_element = element.find_element(By.CLASS_NAME, "price").get_attribute("textContent")
                img_element = element.find_element(By.TAG_NAME, "img").get_attribute("src")
                
                # Agregar validación para evitar duplicados
                if not any(item['Title'] == title_element.text for item in data):
                    data.append({
                        "Title": title_element.text,
                        "Price": price_element,
                        "Image": img_element
                    })
                else:
                    logging.info(f"Duplicado encontrado: {title_element.text}")
                
            except NoSuchElementException as e:
                logging.warning(f"Element not found in the current item: {e}")
    except TimeoutException:
        logging.error("Timeout while waiting for elements to load.")
    
    return data

if __name__ == "__main__":
    url = "https://www.alkosto.com/search/?text=TV"
    driver = configure_driver()

    try:
        logging.info("Iniciando scraping de TVs...")
        tv_data = scrape_tv_data(driver, url)

        if tv_data:
            df = pd.DataFrame(tv_data)
            output_file = "alkosto_tvs.csv"
            df.to_csv(output_file, index=False)
            logging.info(f"Datos guardados en {output_file}.")
        else:
            logging.info("No se encontraron datos para guardar.")
    
    finally:
        logging.info("Cerrando el WebDriver.")
        driver.quit()


In [43]:
df # Dataframe visualization

,Title,Price,Image
0,"TV KALLEY 50"" Pulgadas 127 cm GTV50UHDQV2 4K-U...",$1.299.900,https://www.alkosto.com/medias/7705946478175-0...
1,"TV LG 50"" Pulgadas 126 Cm 50UR7410PSA 4K-UHD L...",$1.599.900,https://www.alkosto.com/medias/8806096079416-0...
2,"TV LG 55"" Pulgadas 139 Cm 55UT8050PSB 4K-UHD L...",$1.999.900,https://www.alkosto.com/medias/8806096082249-0...
3,"TV LG 65"" Pulgadas 165 Cm 65UT7300 4K-UHD LED ...",$2.399.900,https://www.alkosto.com/medias/8806084530950-0...
4,"TV HYUNDAI 43"" Pulgadas 109,2 cm HYLED4322GiM ...",$979.900,https://www.alkosto.com/medias/7707138071775-0...
...,...,...,...
195,Limpiador EMMTECGNOLOGY para pantallas de Tele...,$39.900,https://www.alkosto.com/medias/310Wx310H-maste...
196,Mesa TV INVAL MTV17519 Amaretto,$429.900,https://www.alkosto.com/medias/7707036532507-0...
197,"Instalación de TV 70"" a 90"" incluye Base de In...",$389.900,https://www.alkosto.com/medias/7705946375337-0...
198,"Mesa para TV de 65"" MADERKIT Aurora Gales-Wengue",$439.900,https://www.alkosto.com/medias/7706112746050-0...


## WebScrapping to Amazon (Online Store)

### 4. Configuración del WebDriver y Extracción de Información de TVs

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import logging
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Instalar automáticamente Chromedriver
chromedriver_autoinstaller.install()

# Configurar el WebDriver
options = Options()
options.add_argument("--headless")  # Ejecutar en modo sin cabeza
options.add_argument("--no-sandbox")  # Opcional
options.add_argument("--disable-dev-shm-usage")  # Opcional

driver = webdriver.Chrome(service=Service(), options=options)

def scrape_amazon(url):
    driver.get(url)
    data = []

    while True:
        try:
            # Esperar hasta que los productos estén presentes
            products = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.s-main-slot .s-result-item'))
            )

            for product in products:
                title = price_whole = img_url = "Sin información"

                try:
                    # Obtener el título del producto
                    title_element = product.find_element(By.TAG_NAME, 'h2')
                    title = title_element.text.strip() if title_element else "Sin título"

                    # Filtrar resultados no deseados
                    if "Búsquedas relacionadas" in title or "¿Necesitas ayuda?" in title:
                        continue  # Omitir este producto

                    # Obtener el precio
                    try:
                        price_whole_element = product.find_element(By.CLASS_NAME, 'a-price-whole')
                        price_fraction_element = product.find_element(By.CLASS_NAME, 'a-price-fraction')
                        price_whole = price_whole_element.text + ('.' + price_fraction_element.text if price_fraction_element else '')
                    except NoSuchElementException:
                        price_whole = "Sin precio"

                    # Obtener la imagen
                    try:
                        img_element = product.find_element(By.TAG_NAME, 'img')
                        img_url = img_element.get_attribute("src") if img_element else img_url
                        if "No image" in img_url:  # Verificar si la URL es válida
                            img_url = "Sin imagen"
                    except NoSuchElementException:
                        img_url = "No image"

                    # Agregar validación para evitar duplicados y omitir títulos vacíos
                    if title and not any(item['Title'] == title for item in data):
                        data.append({
                            "Title": title,
                            "Price": price_whole,
                            "Image": img_url
                        })
                    elif not title:
                        logging.info("Título vacío encontrado, se omitirá el elemento.")
                    else:
                        logging.info(f"Duplicado encontrado: {title}")

                except NoSuchElementException as e:
                    pass

            # Intentar obtener el href del botón "Siguiente"
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, '.s-pagination-next')
                next_page_url = next_button.get_attribute("href")
                
                if next_page_url:
                    logging.info(f"Moviéndose a la siguiente página: {next_page_url}")
                    driver.get(next_page_url)  # Cargar la siguiente página
                    WebDriverWait(driver, 10).until(EC.staleness_of(products[0]))  # Esperar que la página cambie
                else:
                    logging.info("No se encontró la URL de la siguiente página.")
                    break  # Termina si no se encuentra la URL de la siguiente página

            except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
                logging.info("No se encontró el botón de siguiente o hubo un error al hacer clic.")
                break

        except TimeoutException:
            logging.error("Timeout while waiting for elements to load.")
            break
    
    return data

# Ejecutar scraping
url_amazon = "https://www.amazon.com/s?k=TV"
tv_data_amazon = scrape_amazon(url_amazon)

# Crear DataFrame y mostrarlo
df_amazon = pd.DataFrame(tv_data_amazon)

# Guardar los datos en un CSV
df_amazon.to_csv("amazon_tvs.csv", index=False)
print("Datos guardados en amazon_tvs.csv")

driver.quit()  # Cerrar el WebDriver



2024-10-01 19:16:46,321 - INFO - Chromedriver is already installed.
2024-10-01 19:16:55,077 - INFO - Moviéndose a la siguiente página: https://www.amazon.com/-/es/s?k=TV&page=2&qid=1727828220&ref=sr_pg_1
2024-10-01 19:16:56,371 - INFO - Duplicado encontrado: Resultados
2024-10-01 19:16:57,176 - INFO - Moviéndose a la siguiente página: https://www.amazon.com/-/es/s?k=TV&page=3&language=es&qid=1727828223&ref=sr_pg_2
2024-10-01 19:16:58,634 - INFO - Duplicado encontrado: Resultados
2024-10-01 19:16:59,607 - INFO - Moviéndose a la siguiente página: https://www.amazon.com/-/es/s?k=TV&page=4&language=es&qid=1727828225&ref=sr_pg_3
2024-10-01 19:17:01,105 - INFO - Duplicado encontrado: Resultados
2024-10-01 19:17:02,092 - INFO - Moviéndose a la siguiente página: https://www.amazon.com/-/es/s?k=TV&page=5&language=es&qid=1727828228&ref=sr_pg_4
2024-10-01 19:17:03,415 - INFO - Duplicado encontrado: Resultados
2024-10-01 19:17:04,430 - INFO - Moviéndose a la siguiente página: https://www.amazon.co

Datos guardados en amazon_tvs.csv


In [33]:
df_amazon # DataFrame Visualization

,Title,Price,Image
0,Resultados,Sin precio,No image
1,Televisión inteligente Amazon Fire TV 2-Series...,Sin precio,https://m.media-amazon.com/images/I/71Nma1KADe...
2,VIZIO Smart TV Full HD 1080p de 40 pulgadas co...,148.00,https://m.media-amazon.com/images/I/81R3dLptKc...
3,SAMSUNG Smart TV Class Crystal UHD 4K DU7200 S...,Sin precio,https://m.media-amazon.com/images/I/71O5nf39wS...
4,Hisense Smart Roku TV de 40 pulgadas Serie A4 ...,159.99,https://m.media-amazon.com/images/I/71cJVrOAVN...
...,...,...,...
208,Sony 50 pulgadas 4K Ultra HD TV Serie X77L: LE...,Sin precio,https://m.media-amazon.com/images/I/71dRtTqxgh...
209,"SAMSUNG HBU8000 HG43BU800NFXZA 43"" Smart LED-L...",517.17,https://m.media-amazon.com/images/I/51AhMHR249...
210,"TV de pantalla plana de 14 pulgadas, pequeño t...",99.99,https://m.media-amazon.com/images/I/714i+xAFwJ...
211,Samsung Monitor inteligente FHD de 32 pulgadas...,Sin precio,https://m.media-amazon.com/images/I/61i+dfL6Dg...


### 5. Data Preparation

In [51]:
import pandas as pd

# Primero,  los valores "sin precio" a NaN para que sea más fácil filtrarlos
df_amazon['Price'] = pd.to_numeric(df_amazon['Price'], errors='coerce')

#Eliminar los NaN en la columna 'price'
df_amazon_clean = df_amazon.dropna(subset=['Price'])

# Mostrar las primeras filas del DataFrame limpio
print(df_amazon_clean.head())
csv_file_path = 'df_amazon_clean.csv'
df_amazon_clean.to_csv(csv_file_path, index=False)


                                                Title   Price  \
2   VIZIO Smart TV Full HD 1080p de 40 pulgadas co...  148.00   
4   Hisense Smart Roku TV de 40 pulgadas Serie A4 ...  159.99   
5   VIZIO Smart TV LED HD 720p de 32 pulgadas con ...  128.00   
7   VIZIO - Smart TV Full HD 1080p de 24 pulgadas ...  124.18   
10  SAMSUNG Televisor inteligente FHD LED de 32 pu...  227.99   

                                                Image  
2   https://m.media-amazon.com/images/I/81R3dLptKc...  
4   https://m.media-amazon.com/images/I/71cJVrOAVN...  
5   https://m.media-amazon.com/images/I/81qgm6blFr...  
7   https://m.media-amazon.com/images/I/81OsEmGZ1d...  
10  https://m.media-amazon.com/images/I/91snjIt0nU...  


### 6. Change US to COP using API

In [55]:
import pandas as pd
import requests

# Ruta del archivo CSV 
file_path = r'C:\Users\JUANPABLOGASCA\Desktop\WebScrapping\df_amazon_clean.csv'  

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(file_path)

# API URL de ExchangeRate-API (con tu propia clave)
api_url = "https://v6.exchangerate-api.com/v6/YOUR_API_KEY/latest/USD"

# Hacer la solicitud a la API
response = requests.get(api_url)

# Imprimir el código de estado y el contenido de la respuesta
print(f"Código de estado: {response.status_code}")
print(f"Contenido de la respuesta: {response.text}")

# Intentar convertir la respuesta a JSON
try:
    data = response.json()
    
    # Extraer la tasa de cambio de USD a COP
    usd_to_cop_rate = data['conversion_rates']['COP']
    print(f"Tasa de cambio actual USD a COP: {usd_to_cop_rate}")

    # Convertir los precios en dólares a pesos colombianos
    df['price_cop'] = df['Price'] * usd_to_cop_rate

    # Redondear los valores a enteros
    df['price_cop'] = df['price_cop'].round(0).astype(int)

    # Mostrar las primeras filas del DataFrame con la nueva columna
    print(df[['Price', 'price_cop']].head())
except requests.JSONDecodeError:
    print("Error al decodificar la respuesta JSON.")



Código de estado: 404
Contenido de la respuesta: <html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>

Error al decodificar la respuesta JSON.


In [56]:
import pandas as pd
import requests

# Ruta del archivo CSV 
file_path = r'C:\Users\JUANPABLOGASCA\Desktop\WebScrapping\df_amazon_clean.csv'  

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(file_path)

# API URL de ExchangeRatesAPI (con tu propia clave)
api_url = "https://open.er-api.com/v6/latest/USD"

# Hacer la solicitud a la API
response = requests.get(api_url)

# Imprimir el código de estado y el contenido de la respuesta
print(f"Código de estado: {response.status_code}")
print(f"Contenido de la respuesta: {response.text}")

# Intentar convertir la respuesta a JSON
try:
    data = response.json()
    
    # Extraer la tasa de cambio de USD a COP
    usd_to_cop_rate = data['rates']['COP']
    print(f"Tasa de cambio actual USD a COP: {usd_to_cop_rate}")

    # Convertir los precios en dólares a pesos colombianos
    df['price_cop'] = df['Price'] * usd_to_cop_rate

    # Redondear los valores a enteros
    df['price_cop'] = df['price_cop'].round(0).astype(int)

    # Mostrar las primeras filas del DataFrame con la nueva columna
    print(df[['Price', 'price_cop']].head())
except requests.JSONDecodeError:
    print("Error al decodificar la respuesta JSON.")


Código de estado: 200
Contenido de la respuesta: {"result":"success","provider":"https://www.exchangerate-api.com","documentation":"https://www.exchangerate-api.com/docs/free","terms_of_use":"https://www.exchangerate-api.com/terms","time_last_update_unix":1727827351,"time_last_update_utc":"Wed, 02 Oct 2024 00:02:31 +0000","time_next_update_unix":1727913901,"time_next_update_utc":"Thu, 03 Oct 2024 00:05:01 +0000","time_eol_unix":0,"base_code":"USD","rates":{"USD":1,"AED":3.6725,"AFN":68.732299,"ALL":88.932561,"AMD":387.303241,"ANG":1.79,"AOA":957.665745,"ARS":971.25,"AUD":1.451352,"AWG":1.79,"AZN":1.700895,"BAM":1.76582,"BBD":2,"BDT":119.498296,"BGN":1.765648,"BHD":0.376,"BIF":2902.613825,"BMD":1,"BND":1.287869,"BOB":6.920679,"BRL":5.451328,"BSD":1,"BTN":83.908458,"BWP":13.122813,"BYN":3.249459,"BZD":2,"CAD":1.35021,"CDF":2845.713226,"CHF":0.84631,"CLP":897.872516,"CNY":7.024875,"COP":4225.143702,"CRC":518.817035,"CUP":24,"CVE":99.552707,"CZK":22.831175,"DJF":177.721,"DKK":6.737346,"DOP

In [58]:
# Final DataSet for further analysis
csv_file_path = 'AmazonTvDataSet.csv'
df.to_csv(csv_file_path, index=False)
df

,Title,Price,Image,price_cop
0,VIZIO Smart TV Full HD 1080p de 40 pulgadas co...,148.00,https://m.media-amazon.com/images/I/81R3dLptKc...,625321
1,Hisense Smart Roku TV de 40 pulgadas Serie A4 ...,159.99,https://m.media-amazon.com/images/I/71cJVrOAVN...,675981
2,VIZIO Smart TV LED HD 720p de 32 pulgadas con ...,128.00,https://m.media-amazon.com/images/I/81qgm6blFr...,540818
3,VIZIO - Smart TV Full HD 1080p de 24 pulgadas ...,124.18,https://m.media-amazon.com/images/I/81OsEmGZ1d...,524678
4,SAMSUNG Televisor inteligente FHD LED de 32 pu...,227.99,https://m.media-amazon.com/images/I/91snjIt0nU...,963291
...,...,...,...,...
135,Jensen JTV1921DC TV LED HD de 19 pulgadas con ...,279.00,https://m.media-amazon.com/images/I/61mBnh3vfY...,1178815
136,JE2612LEDWM Vehículo TV LED de 26 pulgadas; 12...,399.00,https://m.media-amazon.com/images/I/515M0DfLp-...,1685832
137,"SAMSUNG HBU8000 HG43BU800NFXZA 43"" Smart LED-L...",517.17,https://m.media-amazon.com/images/I/51AhMHR249...,2185118
138,"TV de pantalla plana de 14 pulgadas, pequeño t...",99.99,https://m.media-amazon.com/images/I/714i+xAFwJ...,422472


In [1]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
